In [1]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-wc3rzigz/unsloth_63206f41e1fb4ec080f1cdad676632c6
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-wc3rzigz/unsloth_63206f41e1fb4ec080f1cdad676632c6
  Resolved https://github.com/unslothai/unsloth.git to commit 6c534341bb229b136f9504443f0161645d2070c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.8 MB/s eta 0:00:00
  

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Mistral-Nemo-Base-2407",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Mistral patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/165k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.31G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.26M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [4]:
! pip install datasets

In [5]:
from datasets import load_dataset

ds = load_dataset("enimai/MuST-C-fr")

README.md:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/321k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/559k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/275085 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1412 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2632 [00:00<?, ? examples/s]

In [9]:
from datasets import DatasetDict

In [10]:
ds = DatasetDict({
    'train': ds['train'].select(range(5000)),
    'val': ds['validation']
})


In [11]:
def format_sample(sample, eos_token):
    blurb = "\nBelow is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request."
    instruction = "### Instruction: Translate the following text into French."
    input_context = f"### Input:\n{sample.get('en', '')}"
    response = f"### Response:\n{sample.get('fr', '')}"
    end = "### End"
    return "\n\n".join([blurb, instruction, input_context, response, end]) + eos_token

# Apply formatting to dataset
eos_token = tokenizer.eos_token
ds = ds.map(lambda x: {"text": format_sample(x, eos_token)}, num_proc=2)

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1412 [00:00<?, ? examples/s]

In [12]:
from transformers import TrainingArguments
from trl import SFTTrainer
from unsloth import is_bfloat16_supported

In [20]:
training_args = TrainingArguments(
    per_device_train_batch_size=15,
    per_device_eval_batch_size=5,
    gradient_accumulation_steps=5,
    warmup_steps=5,
    max_steps=30,
    learning_rate=2e-4,
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    eval_steps=5,
    evaluation_strategy="steps",
    save_steps=5,
    save_total_limit=2,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
     output_dir="./results"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=training_args,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1412 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [22]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 15 | Gradient Accumulation steps = 5
\        /    Total batch size = 75 | Total steps = 30
 "-____-"     Number of trainable parameters = 57,016,320


Step,Training Loss,Validation Loss
5,1.754000,1.422493
10,0.978600,0.977579
15,0.991000,0.948979
20,1.033700,0.941262
25,0.947600,0.934048
30,0.959600,0.931790


TrainOutput(global_step=30, training_loss=1.2172477920850118, metrics={'train_runtime': 3717.5021, 'train_samples_per_second': 0.605, 'train_steps_per_second': 0.008, 'total_flos': 2.88279845669376e+16, 'train_loss': 1.2172477920850118, 'epoch': 0.4491017964071856})

In [23]:
FastLanguageModel.for_inference(model)

# Define the prompt for translation using the new structure
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Translate the following text into French.

### Input:
{}

### Response:
{}"""

# For inference, you'll leave the output field blank to let the model generate it.
inputs = tokenizer(
    [
        prompt.format(
            '''
           A hilarious comedy filled with laugh-out-loud moments, witty dialogue, and unforgettable characters. The humor is spot-on, making it a perfect pick for anyone in need of a good time.
             ''',  # input - English text
            ""  # output - leave this blank for generation!
        )
    ],
    return_tensors="pt"
)

# Generate the output
outputs = model.generate(**inputs, max_new_tokens = 120, use_cache = True)
tokenizer.batch_decode(outputs)


from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)


<s>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Translate the following text into French.

### Input:

           A hilarious comedy filled with laugh-out-loud moments, witty dialogue, and unforgettable characters. The humor is spot-on, making it a perfect pick for anyone in need of a good time. 
             

### Response:
Une comédie hilarante remplie de moments qui font rire aux éclats, de dialogues intelligents et de personnages inoubliables. L'humour est juste, ce qui en fait un choix parfait pour ceux qui ont besoin de se détendre.

### End</s>
